## Quantinuum tutorial

This notebook will show you how to use pytket to run circuits on the Quantinuum backends. You will need to install `pytket-quantinuum`.

This tutorial assumes you are familiar with creating circuits in `pytket`. See the [pytket notebook](pytket_tutorial.ipynb) for an introduction.

The examples here will go through:
- [Basic usage](#Basic-emulator-usage)
- [Using myqos with Quantinuum](#Using-Myqos)
- [Cost estimation](#Cost-estimation)

More thorough documentation can be found:
- On the [Quantinuum user portal](https://um.qapi.quantinuum.com/user) ('Examples' -> 'docs')
- On the [pytket-quantinuum documentation](https://cqcl.github.io/pytket-quantinuum/api/)
- On the [Quantinuum Machine Info' Confluence page](https://cqc.atlassian.net/wiki/spaces/HQS/pages/920518657/Quantinuum+Machine+Info)
- On the [Quantinuum via Myqos section](https://myqos.com/docs/example_notebooks/quantinuum_examples.html) of the Myqos documentation

### Import the necessary packages

In [1]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as rcj

## Create a circuit

In [2]:
c = Circuit(2).H(0).CX(0, 1).measure_all()

rcj(c)

## Basic usage

In [3]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend('H1-1E', provider='microsoft')
compiled_circuit = backend.get_compiled_circuit(c)
handle = backend.process_circuit(compiled_circuit, n_shots=100)
result = backend.get_result(handle)
counts = result.get_counts()
print(counts)

To sign in:
1) Open a web browser (using any device)
2) Visit https://microsoft.com/devicelogin
3) Enter code 'F3YYZYSDL'
4) Enter your Microsoft credentials
Authentication successful
Counter({(0, 0): 53, (1, 1): 47})


Using a real device works in exactly the same way, just replacing `H1-1E` with `H1-1`. However, ***you must request time on the real devices first***. See the [confluence page](https://cqc.atlassian.net/wiki/spaces/HQS/pages/920518657/Quantinuum+Machine+Info) for information on how to do this.

## Batching circuits

You can batch circuits together so that when it's your turn in the queue, they will run one after the other.

You can do this manually, as shown below, or by uing Myqos' automatic batching feature. See the [Myqos section](#Using-Myqos) below for more details.

In [8]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend('H1-1E', provider='microsoft')

circuits = [c.copy(), c.copy(), c.copy()]
compiled_circuits = backend.get_compiled_circuits(circuits)

handle1 = backend.start_batch(max_batch_cost=100, circuit=compiled_circuits[0], n_shots=100)
handle2 = backend.add_to_batch(handle1, max_batch_cost=100, circuit=compiled_circuits[1], n_shots=100)
handle3 = backend.add_to_batch(handle1, max_batch_cost=100, circuit=compiled_circuits[2], n_shots=100, batch_end=True)

result1 = backend.get_result(handle1)
print(result1.get_counts())

Counter({(0, 0): 51, (1, 1): 48, (1, 0): 1})


Currently there isn't a neat way to do this batching process automatically if you don't want to use Myqos. You could do a workaround such as:

In [11]:
num_circuits = 5
circuits = [c.copy() for _ in range(num_circuits)]
compiled_circuits = backend.get_compiled_circuits(circuits)

# Start batch
h1 = backend.start_batch(max_batch_cost=100, circuit=compiled_circuits[0], n_shots=100)
handles = [h1]

# Add remaining circuits to batch
for i in range(1, num_circuits):
    batch_end = True if i == num_circuits - 1 else False
    h = backend.add_to_batch(h1, max_batch_cost=100, circuit=compiled_circuits[i], n_shots=100, batch_end=batch_end)
    handles.append(h)

results = [backend.get_result(h) for h in handles]
counts = [r.get_counts() for r in results]
print(counts[0])

Counter({(1, 1): 64, (0, 0): 36})


## Using Myqos

Myqos lets you store circuit results in the cloud. See the [Myqos tutorial](myqos_tutorial.ipynb) for more information, as well as the [Myqos documentation](https://myqos.com/docs/).

Myqos automatically supports automatic batching for Quantinuum backends. You just need to pass `attempt_batch=True` to `QuantinuumConfig`. This means that once it's your turn in the queue, the whole batch will run together. See more details on the [Quantinuum via Myqos page](https://myqos.com/docs/example_notebooks/quantinuum_examples.html#Batching) and the [pytket-quantinuum docs](https://cqcl.github.io/pytket-quantinuum/api/index.html).

In [4]:
from pytket.extensions.myqos import Myqos, MyqosBackend, QuantinuumConfig

myqos = Myqos()
myqos.login()
quantinuum_config = QuantinuumConfig(device_name='H1-1E', user_group='DEFAULT', attempt_batching=True)

try:
    experiment = myqos.new_experiment('h_series_test')
except:
    experiment = myqos.get_experiment_by_name('h_series_test')

backend = MyqosBackend(quantinuum_config, experiment)


Started using experiment with name: h_series_test


In [17]:
circuits = [c.copy(), c.copy(), c.copy()]

compiled_circuits = backend.get_compiled_circuits(circuits)
handles = backend.process_circuits(compiled_circuits, n_shots=100)

In [18]:
results = [backend.get_result(h, timeout=None) for h in handles]
counts = [r.get_counts() for r in results]
print(counts)

[Counter({(1, 1): 60, (0, 0): 40}), Counter({(1, 1): 54, (0, 0): 45, (1, 0): 1}), Counter({(0, 0): 50, (1, 1): 49, (1, 0): 1})]


## Cost estimation

The Quantinuum backends have a `.cost` function which returns the HQCs of a circuit. You may need to explicitly provide the `syntax_checker` you need to use.

In [14]:
from pytket.extensions.quantinuum import QuantinuumBackend
emulator_backend = QuantinuumBackend('H1-1E')

In [12]:
emulator_backend.cost(
    emulator_backend.get_compiled_circuit(c), n_shots=100, syntax_checker="H1-1SC"
)

5.66

A more complicated circuit will be more expensive, as will the same circuit with more shots.

In [13]:
emulator_backend.cost(
    emulator_backend.get_compiled_circuit(c), n_shots=5000, syntax_checker="H1-1SC"
)

38.0